In [1]:
%load_ext autoreload
%autoreload 2

# **Environment prep for Colab**


*   download git repo
*   install required pip packages



In [2]:
\from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
!git clone https://github.com/kkahloots/Generative_Models.git
IMG_DIR = '/content/Generative_Models/data/.pokemon'
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
!pip install colorlog==4.0.2

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists
Cloning into 'Generative_Models'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 16171 (delta 0), reused 3 (delta 0), pack-reused 16162
Receiving objects: 100% (16171/16171), 1.00 GiB | 25.91 MiB/s, done.
Resolving deltas: 100% (11012/11012), done.
Checking out files: 100% (1688/1688), done.
/content/Generative_Models
  Found existing installation: colorlog 4.6.2
    Uninstalling colorlog-4.6.2:
      Successfully uninstalled colorlog-4.6.2


In [3]:
%cd /content/Generative_Models/

/content/Generative_Models


# **Connect Drive to save and restore results**

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists


# **Settings**

In [5]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

env: TF_KERAS=1


In [6]:
import tensorflow as tf
print(tf.__version__)

2.3.0


# **only on colab**

In [7]:
!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [8]:
!tar -xvf data/.pokemon/pokemon.tar --directory data/.pokemon/

Pokemon_LMDB/_training/
Pokemon_LMDB/_training/lock.mdb
Pokemon_LMDB/_training/data.mdb
Pokemon_LMDB/_validation/
Pokemon_LMDB/_validation/lock.mdb
Pokemon_LMDB/_validation/data.mdb
Pokemon_LMDB/meta_info.json


# **Dataset loading**

In [13]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [14]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [15]:
from transformation.lmdb_transformer import LmdbTransformer

In [21]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

/content/Generative_Models/transformation/file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [22]:
import numpy as np
_instance_scale=1.0
for data in val_generator:
    print(np.max(data['images']))
    break
    #print(data.keys())

1.0


In [23]:
data['images'][0].shape

TensorShape([400, 400, 3])

In [24]:
data.keys()

dict_keys(['images', 'label'])

# **Model's Layers definition**

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [ ]:
latents_dim

100

In [ ]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),

            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# **Model definition**

In [ ]:
model_name = dataset_name+'AE_Dense_reconst_ell'
experiments_dir='/content/drive/My Drive/Results/experiments'+sep_local+model_name

In [ ]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [ ]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist

In [ ]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [ ]:
create_if_not_exist(_restore)
_restore

'/content/drive/My Drive/Results/experiments/pokemonAE_Dense_reconst_ell/var_save_dir'

In [ ]:
#to restore trained model, set filepath=_restore

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...
  DEBUG    | Restore old models ...


  WARNING  | None
  WARNING  | None


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1666)              168266    
_________________________________________________________________
dense_5 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_6 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_7 (Dense)              (None, 30000)             50010000  
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0

In [ ]:
ae.compile()

Model: "pokemonAE_Dense_reconst_ell"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 100)               50003250  
_________________________________________________________________
generative (Functional)      (None, 100, 100, 3)       55742822  
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 105,746,072
Trainable params: 105,745,866
Non-trainable params: 206
_________________________________________________________________
None


In [ ]:
ae.output_shape

(None, 100, 100, 3)

# **Callbacks**

In [ ]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-15, 
    patience=10, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'/content/drive/My Drive/Results/experiments/pokemonAE_Dense_reconst_ell/csv_dir/pokemonAE_Dense_reconst_ell.csv'

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [ ]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# **Model training**

In [ ]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000

Epoch 00001: loss improved from inf to 6677.68066, saving model to /content/drive/My Drive/Results/experiments/pokemonAE_Dense_reconst_ell/var_save_dir
5000/5000 - 1224s - loss: 6677.6807 - psnr: 11.3392 - total_variation: 86505.9531 - ssim_multiscale: 4.1016 - sharpdiff: 5.3047 - mean_absolute_error: 0.1997 - mean_squared_error: 0.0756 - val_loss: 1376.8918 - val_psnr: 20.7102 - val_total_variation: 32626.3965 - val_ssim_multiscale: 8.8171 - val_sharpdiff: 11.6103 - val_mean_absolute_error: 0.0652 - val_mean_squared_error: 0.0099
Epoch 2/1000000

Epoch 00002: loss improved from 6677.68066 to 4477.53613, saving model to /content/drive/My Drive/Results/experiments/pokemonAE_Dense_reconst_ell/var_save_dir
5000/5000 - 1227s - loss: 4477.5361 - psnr: 12.8827 - total_variation: 102325.2266 - ssim_multiscale: 6.3236 - sharpdiff: 5.1719 - mean_absolute_error: 0.1395 - mean_squared_error: 0.0523 - val_loss: 392.0399 - val_psnr: 26.4700 - val_total_variation: 31641.5137 - val_s

# **Model evaluation**

## **inception_score**

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=100)
print(f'inception_score mean: {is_mean.compute()}, sigma: {is_sigma.compute()}')

87916544/87910968 [==============================] - 0s 0us/step
calculating the inception_score mean ...


100it [48:54, 29.21s/it]

calculating the inception_score sigma ...


/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
100it [47:45, 27.54s/it]

inception_score mean: [9.64356898e+140], sigma: 1.3710762563482616e+284


## **Frechet_inception_distance**

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, train_generator.as_numpy_iterator(), 
                                       tolerance_threshold=1e-6, 
                                       max_iteration=100, 
                                       batch_size=32)
print(f'frechet inception distance: {fis_score.compute()}')

calculating the inception images sigma ...


0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
1it [00:56, 56.68s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
2it [02:27, 66.91s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
3it [03:57, 73.90s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
4it [05:27, 78.68s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
5it [06:59, 82.75s/it]/usr/local/lib/python3.6/dist-packages/dask/array/ro

calculating the inception images mean ...


0it [00:00, ?it/s]

calculating the generated images sigma ...


/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
1it [00:52, 52.54s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
2it [02:18, 62.60s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
3it [03:44, 69.44s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
4it [05:09, 74.22s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 205
  axes=(left_axes, right_axes),
5it [06:35, 77.84s/it]/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:272: Per

calculating the generated images mean ...


100it [00:41,  2.19it/s]

## **perceptual_path_length_score**

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, train_generator.as_numpy_iterator(), 
                                              tolerance_threshold=1e-6, 
                                              max_iteration=100, 
                                              batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

100it [01:30,  1.09it/s]

perceptual path length score: 5102.37548828125


## **precision score**

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae,  train_generator.as_numpy_iterator(), 
                                   tolerance_threshold=1e-6, 
                                   max_iteration=100)
print(f'precision score: {_precision_score}')

58892288/58889256 [==============================] - 0s 0us/step


/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
                  

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00294042s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00202656s
precision score: [0.]


## **recall score**

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, train_generator.as_numpy_iterator(), tolerance_threshold=1e-6, max_iteration=100)
print(f'recall score: {_recall_score}')

0it [00:00, ?it/s]/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
                  

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00152254s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00161433s
recall score: [1.]



# **Image Generation**


## **image reconstruction**


### **Training dataset**

In [ ]:
from training.generators.from_images.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:00, 57.48it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 42.23it/s]


## **with Randomness**

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:00, 35.63it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 36.17it/s]


### **Complete Randomness**

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

32it [00:00, 59.44it/s]


In [ ]:
from training.generators.from_images.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 83.51it/s]
0it [00:00, ?it/s]

10it [00:00, 78.02it/s]
10it [00:00, 89.72it/s]
0it [00:00, ?it/s]

10it [00:00, 74.41it/s]
100%|██████████| 4/4 [00:00<00:00,  6.62it/s]
10it [00:00, 92.47it/s]
